In [1]:
import sys
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import time

sys.path.append('../')

### Binary

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/home-credit-default-risk/train.parquet')
target_col = 'target'
index_col = 'sk_id_curr'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]
with open('/www/dslib/spark_sota_modeling/dataset/home-credit-default-risk/categorical_features.txt', 'r') as f:
    categorical_features = [line.strip() for line in f.readlines()]
len(categorical_features)

16

In [3]:
from models.estimators.tabnet_estimator import TabNetBinary
from models.estimators.cemlp_estimator import CatEmbMLPBinary
# model = CatEmbMLPBinary(
#     cat_emb_dim=2,
#     hidden_dims=[256, 128],
#     activation='swish',
#     dropout=0.6,
#     initialization='xavier_uniform',
#     normalization='ghost_batch', # ghost_batch
#     batch_size=1024,
#     virtual_batch_size=128,
#     momentum=0.9,
#     epochs=100,
#     learning_rate=0.001,
#     weight_decay=1e-5,
#     early_stopping_patience=10,
#     scale_numerical=True,
#     scale_method='standard',
#     n_bins=10,
#     verbose=True,
#     random_state=42,
#     lr_scheduler_patience=3,
#     lr_scheduler_factor=0.6,
#     dynamic_emb_size=False,
#     feature_dropout=0.1,
#     use_self_attention=False,
#     num_attention_heads=4,
# )
model = TabNetBinary(
    batch_size=1024,
    cat_emb_dim=1,
    dynamic_emb_size=True,
    n_a=32,
    n_d=32,
    n_glu_layers=4,
    n_steps=7,
    decision_dim=16,
    dropout=0.7,
    early_stopping_patience=10,
    epochs=200,
    gamma=1.5,
    lambda_sparse=0.0001,
    learning_rate=0.001,
    momentum=0.9,
    random_state=42,
    reducelronplateau_factor=0.6,
    reducelronplateau_patience=3,
    verbose=True,
    virtual_batch_size=128,
    weight_decay=1e-5,
)
model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='roc_auc', mode='max', cat_features=categorical_features)
y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
roc_auc_score(y_test, y_pred_proba[:,1])

Начинаем обучение на cuda...


Training:   0%|          | 0/241 [00:00<?, ?it/s]

Validation:   0%|          | 0/61 [00:00<?, ?it/s]

Epoch 1/200, Train loss: 0.3685, Train roc_auc: 0.5754, Val loss: 0.2866, Val roc_auc: 0.7484
Сохраняем лучшую модель с метрикой roc_auc: 0.7484


Training:   0%|          | 0/241 [00:00<?, ?it/s]

...

Training:   0%|          | 0/241 [00:00<?, ?it/s]

Validation:   0%|          | 0/61 [00:00<?, ?it/s]

Epoch 43/200, Train loss: 0.2343, Train roc_auc: 0.7962, Val loss: 0.2437, Val roc_auc: 0.7770
Останавливаем обучение из-за отсутствия улучшений в течение 10 эпох
Загружена лучшая модель


Predicting:   0%|          | 0/61 [00:00<?, ?it/s]

0.777291327594837

In [ ]:
from models.estimators.tabnet_estimator import TabNetBinary
from models.estimators.cemlp_estimator import CatEmbMLPBinary

for learning_rate in [0.01, 0.001]:
    for weight_decay in [1e-4, 1e-5, 1e-6, 1e-7]:
        start_time = time.time()
        # model = TabNetBinary(
        #     batch_size=1024,
        #     cat_emb_dim=4,
        #     dynamic_emb_size=True,
        #     n_a=16,
        #     n_d=16,
        #     decision_dim=8,
        #     dropout=0.6,
        #     early_stopping_patience=10,
        #     epochs=200,
        #     gamma=1.5,
        #     lambda_sparse=0.0001,
        #     learning_rate=learning_rate,
        #     momentum=0.9,
        #     n_glu_layers=3,
        #     n_steps=4,
        #     random_state=42,
        #     reducelronplateau_factor=0.6,
        #     reducelronplateau_patience=3,
        #     verbose=False,
        #     virtual_batch_size=128,
        #     weight_decay=weight_decay,
        # )
        model = CatEmbMLPBinary(
            cat_emb_dim=1,
            hidden_dims=[256, 128],
            activation='swish',
            # leaky_relu_negative_slope=0.1,
            dropout=0.5,
            batch_size=1024,
            normalization='ghost_batch',
            virtual_batch_size=128,
            momentum=0.9,
            initialization='xavier_uniform',
            # constant_value=0.001,
            epochs=100,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            early_stopping_patience=10,
            random_state=42,
            lr_scheduler_patience=3,
            lr_scheduler_factor=0.6,
            dynamic_emb_size=True,
            feature_dropout=0.0,
            verbose=False,
            use_self_attention=False,
        )

        model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='roc_auc', mode='max', 
                  cat_features=categorical_features)
        y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
        formatted = time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))
        print(
            f"learning_rate: {learning_rate}, "
            f"weight_decay: {weight_decay}, "
            f"ROC AUC: {roc_auc_score(y_test, y_pred_proba[:,1]):.4f}, "
            f"Time: {formatted}"
        )

### Multiclass

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/train.parquet')
target_col = 'cover_type'
train, test = train_test_split(train, test_size=0.2, random_state=42, stratify=train[target_col])
X_train = train.drop(columns=[target_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col])
y_test = test[target_col]
with open('/www/dslib/spark_sota_modeling/dataset/forest-cover-type/categorical_features.txt', 'r') as f:
    categorical_features = [line.strip() for line in f.readlines()]
len(categorical_features)

44

In [ ]:
from models.estimators.tabnet_estimator import TabNetMulticlass

model = TabNetMulticlass(
    verbose=True,
    n_classes=train[target_col].nunique(),
    dropout=0.1,
    batch_size=16384,
    virtual_batch_size=256,
    learning_rate=0.05,
    epochs=1000,
    dynamic_emb_size=False,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=categorical_features)

y_pred = model.predict(X_test, cat_features=categorical_features)
y_pred_proba = model.predict_proba(X_test, cat_features=categorical_features)
accuracy_score(y_test, y_pred)

Начинаем обучение на cuda...


Training:   0%|          | 0/29 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/1000, Train loss: 0.8019, Train accuracy: 0.6611, Val loss: 0.6107, Val accuracy: 0.7399
Сохраняем лучшую модель с метрикой accuracy: 0.7399


Training:   0%|          | 0/29 [00:00<?, ?it/s]

...

Epoch 67/1000, Train loss: 0.2149, Train accuracy: 0.9112, Val loss: 0.1965, Val accuracy: 0.9195
Нет улучшения в течение 1 эпох


Training:   0%|          | 0/29 [00:00<?, ?it/s]

Validation:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 68/1000, Train loss: 0.2155, Train accuracy: 0.9111, Val loss: 0.1776, Val accuracy: 0.9294
Сохраняем лучшую модель с метрикой accuracy: 0.9294


Training:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
# 0.8479471270104902
# 0.9272996394241112

### Regression

In [2]:
train = pd.read_parquet('/www/dslib/spark_sota_modeling/dataset/allstate-claims-severity/train.parquet')
target_col = 'loss'
index_col = 'id'
train, test = train_test_split(train, test_size=0.2, random_state=42)
X_train = train.drop(columns=[target_col, index_col])
y_train = train[target_col]
X_test = test.drop(columns=[target_col, index_col])
y_test = test[target_col]

In [ ]:
from models.nn.tabnet import TabNetRegressor
model = TabNetRegressor(
    verbose=True,
    dropout=0.3,
    lambda_sparse=0.001,
    learning_rate=0.0015,
    epochs=200,
    early_stopping_patience=20,
    n_glu_layers=2,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='mae', mode='min')

y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

In [ ]:
# Убрать секцию train в конфиге
# Добавить полный список гиперпараметров для всех моделей в конфиг